In [32]:
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import serial
import tdwf
import time
import os

#-[Configurazione seriale]-----------------------------------------------------
ser = serial.Serial()
ser.port = "COM7"
ser.baudrate = 115200
ser.timeout = 1
ser.rts = False
ser.dtr = False
ser.open()

def query(message, flagR=True, flagW=True):
    if flagW:    
        message += '\n'
        ser.write(message.encode('utf-8'))
    if flagR:
        res = ser.readline().decode('utf-8')
        return res


for i in range(12):
    response = ser.readline().decode('utf-8')
    print(response[:-1])


#-[Configurazione AD2]-------------------------------------------------------
ad2 = tdwf.AD2(iconfig=3)

ad2.vdd = +5
ad2.power(True)

#   2. Configurazione generatore di funzioni
wgen = tdwf.WaveGen(ad2.hdwf)
wgen.w1.config(ampl = 5, offs= 0, func=tdwf.funcSine, freq = 4e4)
wgen.w1.start()

#   3. Configurazione oscilloscopio
scope = tdwf.Scope(ad2.hdwf)
scope.fs=5e5
scope.npt=16384
scope.ch1.rng = 10
scope.ch2.rng = 10
scope.trig(True, level = 1, sour = tdwf.trigsrcCh1, delay = 0.04)


#-[Saving pipeline]-----------------------------------------------------
Directory = f"doppler_{time.strftime('%Y%m%d_%H%M%S')}" # directory name based on the current date and time
parent_dir = "../../Data"
path = os.path.join(parent_dir, Directory) # creating the directory
os.mkdir(path)
print(f"Data will be saved in {path}\n")
log_file = os.path.join(path, f"log.txt")   #creating the log file













Dispositivo #1 [SN:210321ABE62D, hdwf=1] connesso!
Configurazione #3
Data will be saved in ../../Data\doppler_20250509_102751



In [33]:

rangeX = [0, 80000]

tt, tx, rx = [], [], []

res = query("tx="+str(rangeX[1]))
print(res[:-1])
time.sleep(.5)

posX = int(query("x?").split('=')[1])
first = True
while(posX < 0.95 * rangeX[1]):
    posX = int(query("x?").split('=')[1])
    print(posX)
    scope.sample()
    tt = tt + scope.time.vals.tolist()
    tx = tx + scope.ch1.vals.tolist()
    rx = rx + scope.ch2.vals.tolist()
#    tt.append(scope.time.vals.tolist())
#    tx.append(scope.ch1.vals.tolist())
#    rx.append(scope.ch2.vals.tolist())
res = query("tx=0")
print(res[:-1])

info = f"Doppler Shift measurement - X = {rangeX[0]:5.0f} -> {rangeX[1]:5.0f} - vX = {query("sx?").split('=')[1] } \n"

data_to_save = np.c_[tt, tx, rx]
info = info + f"time [s]\tch1 [V]\tch2 [V]\n"


filename = f"dopplerShift-{rangeX[0]:.0f}-{rangeX[1]:.0f}"

np.savetxt(path+'/'+filename+".txt", data_to_save, delimiter="\t", header=info, fmt="%s")



TX=80000
1421
1740
2069
2394
2715
3048
3352
3683
4008
4341
4669
4996
5336
5671
5993
6338
6675
7025
7352
7704
8032
8401
8768
9134
9496
9848
10233
10613
11009
11411
11815
12229
12616
13000
13390
13780
14171
14561
14947
15325
15712
16090
16494
16877
17246
17611
18005
18418
18823
19251
19669
20093
20480
20875
21282
21682
22079
22487
22911
23338
23766
24209
24651
25180
25633
26041
26461
26894
27319
27775
28229
28684
29121
29586
30029
30477
30918
31359
31794
32269
32723
33184
33651
34090
34532
34996
35472
35947
36430
36902
37402
37875
38372
38865
39352
39840
40321
40809
41402
42073
42558
43118
43661
44234
44807
45407
45968
46527
47036
47563
48091
48634
49162
49648
50191
50727
51266
51823
52411
52976
53484
54034
54568
55101
55669
56241
56793
57346
57911
58444
59062
59644
60273
60873
61452
62137
62741
63351
63929
64522
65108
65711
66286
66832
67422
67996
68587
69184
69815
70540
71154
71732
72332
72940
73529
74134
74747
75365
76007
TX=0


In [25]:
res = query(input('>> '))
print(res[:-1])

X=0


In [35]:
len(tt)

2605056

In [37]:
np.mean(np.diff(tt))

np.float64(1.2577853442633648e-08)

In [30]:
ser.close()

In [31]:
ad2.close()

Dispositivo disconnesso.
